In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import numpy as np
import pandas as pd
import datetime
from stocks import StockTrades
import matplotlib.pyplot as plt
from positionslib import read_positions_file


In [26]:
!grep -v "as of" /Users/rameshsubrahmanyam/Desktop/all.csv > /Users/rameshsubrahmanyam/Desktop/all_filt.csv


In [27]:
def find_key(key, dct, default=None):
    # Filter keys in the dictionary that are less than or equal to key
    valid_keys = [k for k in dct if k >= key]
    
    if not valid_keys:
        # If no valid keys found, return None 
        return default
    
    # Return the pnl value corresponding to the latest date
    return dct[max(valid_keys)]


In [28]:
def compute_stock_pnl(symbol, start_dates, end_date, price, qty):
    start_dates=list(sorted(start_dates))

    trades = StockTrades(symbol, start_dates[0], end_date)
    trades.read_data("/Users/rameshsubrahmanyam/Desktop/all_filt.csv")
    # Access the filtered DataFrame
    if trades.df is not None:
        pass    
    else:
        print("No data matching the criteria.")
    
    dct=trades.find_pnl(qty, price)
    pnls=[]
    for dt in start_dates:
        dt1=pd.to_datetime(dt, format='%m/%d/%Y')
        pnls.append(find_key(dt1, dct, default=np.NaN))
            # find the latest date in the keys of dct that precedes dt
            # and save the corresponding pnl in out_lst
            # if there is no such date, then store NaN
    return pnls

symbol = "AMD"
# date_format = "%m/%d/%Y"

# # Use datetime.strptime() to parse the date string into a datetime object
# # start_date = datetime.datetime.strptime("01/01/2023", date_format).date()
# # end_date = datetime.datetime.strptime("04/17/2024", date_format).date()

current_price=79.19
start_date = "01/01/2024"
end_date = "05/20/2024"

pos=0
_ =compute_stock_pnl(symbol, [start_date], end_date, current_price, pos)




In [29]:

start_date = "01/01/2024"
end_date = "05/20/2024"
df=read_positions_file("/Users/rameshsubrahmanyam/Desktop/pos.csv")
df=df.loc[df["Security Type"] != "Option", ["Symbol", "Price", "Quantity"]]
symbols=["GLD", "ARKK", "VNQ", "BIDU", "CBRL", "XBI", "ITB", "FI", "DIA", "CRWD",
          "JACK", "XHB",  "XLK", "XLV",  "VO", "XLY", "CRM", "ADSK", "AMD", "XOP", "HD", "LULU"]
pnls=[]
for symbol in sorted(list(set(symbols))):
    #locate the row with Syjmbol == symbol; 

    try:
        x=df.loc[df.Symbol == symbol]
        if len(x) ==0:
            p, q=0,0
        else:
            p, q=x.Price.iloc[0], x.Quantity.iloc[0]
        # print(symbol, p, q)
        
        pnl=compute_stock_pnl(symbol, [start_date], end_date, p, q)
        pnls.append([symbol, p, q] + pnl)
    except:
        print("Skipping", symbol)
out_df=pd.DataFrame(pnls, columns=["symbol", "price", "qty", "pnl"])
print("Total pnl", out_df.pnl.sum())
out_df

                                                                            



Total margin 314384
Total pnl 1348.9500000000103


,symbol,price,qty,pnl
0,ADSK,0.00,0,-0.76
1,AMD,0.00,0,-125.44
2,ARKK,43.61,-10,-15.85
3,BIDU,0.00,0,-43.73
4,CBRL,0.00,0,27.05
5,CRM,0.00,0,NaN
6,CRWD,0.00,0,-32.66
7,DIA,0.00,0,-55.68
8,FI,0.00,0,49.17
9,GLD,0.00,0,1597.16
